# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [4]:
view_sentence_range = (12, 20)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 12 to 20:
india is rainy during june , and it is sometimes warm in november .
that cat was my most loved animal .
he dislikes grapefruit , limes , and lemons .
her least liked fruit is the lemon , but his least liked is the grapefruit .
california is never cold during february , but it is sometimes freezing in june .
china is usually pleasant during autumn , and it is usually quiet in october .
paris is never freezing during november , but it is wonderful in october .
the united states is never rainy during january , but it is sometimes mild in october .

French sentences 12 to 20:
inde est pluvieux en juin , et il est parfois chaud en novembre .
ce chat était mon animal le plus aimé .
il n'aime pamplemousse , citrons verts et les citrons .
son fruit est moins aimé le citron , mais son moins aimé est le pamplemousse .
californie 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [5]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param showource_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # spit line-by-line since each sentence is on separate line
    source_words = source_text.split('\n')
    target_words = target_text.split('\n')
#     print(len(source_text))
#     print(source_words[:5])
#     print(target_words[:5])
    
    # for each line convert words into array of Int
    source_ids = [[source_vocab_to_int[word] for word in words.split()] for words in source_words]
    target_ids = [[target_vocab_to_int[word] for word in words.split()] for words in target_words]
    
    # add EOS int to end of targets only
    target_ids = [target + [target_vocab_to_int['<EOS>']] for target in target_ids]
    
#     print(len(source_ids))
#     print(len(target_ids))
    return source_ids, target_ids

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [3]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    _input = tf.placeholder(tf.int32, [None, None], name = "input")
    _targets = tf.placeholder(tf.int32, [None, None], name = "targets")
    
    _learning_rate = tf.placeholder(tf.float32, name ="learning_rate")
    _keep_prob = tf.placeholder(tf.float32, name = "keep_prob")
    
    _target_seq_len = tf.placeholder(tf.int32,[None], name = "target_sequence_length")
    _max_target_seq_len =  tf.reduce_max(_target_seq_len, name = "max_target_len")
    _source_target_seq_len = tf.placeholder(tf.int32, [None], name = "source_sequence_length")
    
    return _input, _targets, _learning_rate, _keep_prob, _target_seq_len, _max_target_seq_len, _source_target_seq_len


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [4]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    popped_last_slice = tf.strided_slice(target_data, begin=[0,0], end=[batch_size, -1], strides=[1,1])
    go_inputs = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), popped_last_slice], 1)
    
    return go_inputs

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [5]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    embeded = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    cell_initializer = tf.random_uniform_initializer(-0.1, 0.1, seed=2)
    make_cell = lambda size: tf.contrib.rnn.LSTMCell(size,initializer=cell_initializer)
    cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    cell = tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob)
    output, state = tf.nn.dynamic_rnn(cell,embeded,sequence_length=source_sequence_length, dtype=tf.float32)

    # TODO: Implement Function
    return output, state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [6]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    
    helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                               sequence_length=target_sequence_length, 
                                               time_major=False)

    
    decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                              helper, 
                                              encoder_state,
                                              output_layer)

    output, _ = tf.contrib.seq2seq.dynamic_decode(
        decoder=decoder,
        output_time_major=False,
        impute_finished=True,
        maximum_iterations=max_summary_length)
        
    return output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [7]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    go_constant = tf.constant([start_of_sequence_id], dtype=tf.int32)
    start_token = tf.tile(go_constant,[batch_size], name="start_token")
    helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                      start_token,
                                                      end_of_sequence_id)
    
    decoder = tf.contrib.seq2seq.BasicDecoder(cell=dec_cell,
                                              helper=helper,
                                              initial_state=encoder_state,
                                              output_layer=output_layer)
    
    output, _ = tf.contrib.seq2seq.dynamic_decode(decoder=decoder, 
                                                  impute_finished=True, 
                                                  maximum_iterations=max_target_sequence_length)
    return output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [8]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """

    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    cell_initializer = tf.random_uniform_initializer(-0.1, 0.1, seed=2)
    make_cell = lambda size: tf.contrib.rnn.LSTMCell(size,initializer=cell_initializer)
    cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    cell = tf.contrib.rnn.DropoutWrapper(cell, input_keep_prob=keep_prob)
    
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))

    kernal_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1)
    output_layer = Dense(target_vocab_size,kernel_initializer=kernal_initializer)
    
    with tf.variable_scope("decode"):
        train_logits = decoding_layer_train(encoder_state=encoder_state,
                                            dec_cell=cell,
                                            dec_embed_input=dec_embed_input,
                                            target_sequence_length=target_sequence_length,
                                            max_summary_length=max_target_sequence_length,
                                            output_layer=output_layer,
                                            keep_prob=keep_prob)

    with tf.variable_scope("decode", reuse=True):
        infer_logits = decoding_layer_infer(encoder_state=encoder_state,
                                            dec_cell=cell,
                                            dec_embeddings=dec_embeddings,
                                            start_of_sequence_id=target_vocab_to_int['<GO>'], 
                                            end_of_sequence_id=target_vocab_to_int['<EOS>'],
                                            max_target_sequence_length=max_target_sequence_length, 
                                            vocab_size=target_vocab_size,
                                            output_layer=output_layer, 
                                            batch_size=batch_size,
                                            keep_prob=keep_prob)
    return train_logits, infer_logits



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [9]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    enc_output, enc_state = encoding_layer(input_data,
                                   rnn_size,
                                   num_layers,
                                   keep_prob,
                                   source_sequence_length,
                                   source_vocab_size,
                                   enc_embedding_size)
    
    dec_input = process_decoder_input(target_data,
                                       target_vocab_to_int,
                                       batch_size)
    
    training, inference = decoding_layer(dec_input,
                                         enc_state,
                                         target_sequence_length,
                                         max_target_sentence_length,
                                         rnn_size,
                                         num_layers,
                                         target_vocab_to_int,
                                         target_vocab_size,
                                         batch_size,
                                         keep_prob,
                                         dec_embedding_size)
    return training, inference


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [19]:
# Number of Epochs
epochs = 6
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 128
# Number of Layers
num_layers = 3
# Embedding Size
# vocab size is 358
encoding_embedding_size = 400
decoding_embedding_size = 400
# Learning Rate
learning_rate = 0.005
# Dropout Keep Probability
keep_probability = 0.6
display_step = 5


### Build the Graph
Build the graph using the neural network you implemented.

In [11]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    5/1077 - Train Accuracy: 0.2781, Validation Accuracy: 0.3232, Loss: 3.6249
Epoch   0 Batch   10/1077 - Train Accuracy: 0.2734, Validation Accuracy: 0.3693, Loss: 3.5040
Epoch   0 Batch   15/1077 - Train Accuracy: 0.3102, Validation Accuracy: 0.3732, Loss: 3.1653
Epoch   0 Batch   20/1077 - Train Accuracy: 0.3105, Validation Accuracy: 0.3711, Loss: 3.0182
Epoch   0 Batch   25/1077 - Train Accuracy: 0.3348, Validation Accuracy: 0.3945, Loss: 2.9873
Epoch   0 Batch   30/1077 - Train Accuracy: 0.3586, Validation Accuracy: 0.4226, Loss: 2.9136
Epoch   0 Batch   35/1077 - Train Accuracy: 0.3609, Validation Accuracy: 0.3991, Loss: 2.6469
Epoch   0 Batch   40/1077 - Train Accuracy: 0.3895, Validation Accuracy: 0.4506, Loss: 2.5668
Epoch   0 Batch   45/1077 - Train Accuracy: 0.3410, Validation Accuracy: 0.3977, Loss: 2.7084
Epoch   0 Batch   50/1077 - Train Accuracy: 0.3883, Validation Accuracy: 0.4474, Loss: 2.5219
Epoch   0 Batch   55/1077 - Train Accuracy: 0.4043, Validati

Epoch   0 Batch  445/1077 - Train Accuracy: 0.5736, Validation Accuracy: 0.5987, Loss: 0.5777
Epoch   0 Batch  450/1077 - Train Accuracy: 0.5867, Validation Accuracy: 0.5948, Loss: 0.5490
Epoch   0 Batch  455/1077 - Train Accuracy: 0.6158, Validation Accuracy: 0.6112, Loss: 0.5303
Epoch   0 Batch  460/1077 - Train Accuracy: 0.5875, Validation Accuracy: 0.6278, Loss: 0.5560
Epoch   0 Batch  465/1077 - Train Accuracy: 0.6312, Validation Accuracy: 0.6243, Loss: 0.5693
Epoch   0 Batch  470/1077 - Train Accuracy: 0.5929, Validation Accuracy: 0.6254, Loss: 0.5758
Epoch   0 Batch  475/1077 - Train Accuracy: 0.6422, Validation Accuracy: 0.6278, Loss: 0.5331
Epoch   0 Batch  480/1077 - Train Accuracy: 0.6308, Validation Accuracy: 0.6058, Loss: 0.5472
Epoch   0 Batch  485/1077 - Train Accuracy: 0.6152, Validation Accuracy: 0.6147, Loss: 0.5465
Epoch   0 Batch  490/1077 - Train Accuracy: 0.5922, Validation Accuracy: 0.6183, Loss: 0.5533
Epoch   0 Batch  495/1077 - Train Accuracy: 0.6008, Validati

Epoch   0 Batch  885/1077 - Train Accuracy: 0.7212, Validation Accuracy: 0.6634, Loss: 0.2803
Epoch   0 Batch  890/1077 - Train Accuracy: 0.7414, Validation Accuracy: 0.6779, Loss: 0.3189
Epoch   0 Batch  895/1077 - Train Accuracy: 0.6578, Validation Accuracy: 0.6460, Loss: 0.3414
Epoch   0 Batch  900/1077 - Train Accuracy: 0.7078, Validation Accuracy: 0.6740, Loss: 0.3302
Epoch   0 Batch  905/1077 - Train Accuracy: 0.6934, Validation Accuracy: 0.6719, Loss: 0.2915
Epoch   0 Batch  910/1077 - Train Accuracy: 0.6912, Validation Accuracy: 0.6886, Loss: 0.3041
Epoch   0 Batch  915/1077 - Train Accuracy: 0.6772, Validation Accuracy: 0.6985, Loss: 0.3042
Epoch   0 Batch  920/1077 - Train Accuracy: 0.7234, Validation Accuracy: 0.7116, Loss: 0.3055
Epoch   0 Batch  925/1077 - Train Accuracy: 0.7076, Validation Accuracy: 0.6808, Loss: 0.2806
Epoch   0 Batch  930/1077 - Train Accuracy: 0.7277, Validation Accuracy: 0.7106, Loss: 0.2958
Epoch   0 Batch  935/1077 - Train Accuracy: 0.7023, Validati

Epoch   1 Batch  250/1077 - Train Accuracy: 0.7983, Validation Accuracy: 0.7841, Loss: 0.1173
Epoch   1 Batch  255/1077 - Train Accuracy: 0.8402, Validation Accuracy: 0.7976, Loss: 0.1479
Epoch   1 Batch  260/1077 - Train Accuracy: 0.8322, Validation Accuracy: 0.7837, Loss: 0.1231
Epoch   1 Batch  265/1077 - Train Accuracy: 0.8094, Validation Accuracy: 0.7770, Loss: 0.1254
Epoch   1 Batch  270/1077 - Train Accuracy: 0.8027, Validation Accuracy: 0.8036, Loss: 0.1460
Epoch   1 Batch  275/1077 - Train Accuracy: 0.8084, Validation Accuracy: 0.7862, Loss: 0.1373
Epoch   1 Batch  280/1077 - Train Accuracy: 0.8203, Validation Accuracy: 0.7756, Loss: 0.1381
Epoch   1 Batch  285/1077 - Train Accuracy: 0.8393, Validation Accuracy: 0.8008, Loss: 0.1453
Epoch   1 Batch  290/1077 - Train Accuracy: 0.8426, Validation Accuracy: 0.8040, Loss: 0.1530
Epoch   1 Batch  295/1077 - Train Accuracy: 0.8536, Validation Accuracy: 0.7887, Loss: 0.1548
Epoch   1 Batch  300/1077 - Train Accuracy: 0.8655, Validati

Epoch   1 Batch  690/1077 - Train Accuracy: 0.9059, Validation Accuracy: 0.8580, Loss: 0.0625
Epoch   1 Batch  695/1077 - Train Accuracy: 0.8891, Validation Accuracy: 0.8505, Loss: 0.0730
Epoch   1 Batch  700/1077 - Train Accuracy: 0.8809, Validation Accuracy: 0.8327, Loss: 0.0638
Epoch   1 Batch  705/1077 - Train Accuracy: 0.9141, Validation Accuracy: 0.8473, Loss: 0.0785
Epoch   1 Batch  710/1077 - Train Accuracy: 0.8957, Validation Accuracy: 0.8516, Loss: 0.0606
Epoch   1 Batch  715/1077 - Train Accuracy: 0.8738, Validation Accuracy: 0.8594, Loss: 0.0789
Epoch   1 Batch  720/1077 - Train Accuracy: 0.8779, Validation Accuracy: 0.8377, Loss: 0.0799
Epoch   1 Batch  725/1077 - Train Accuracy: 0.9159, Validation Accuracy: 0.8391, Loss: 0.0543
Epoch   1 Batch  730/1077 - Train Accuracy: 0.8863, Validation Accuracy: 0.8636, Loss: 0.0856
Epoch   1 Batch  735/1077 - Train Accuracy: 0.8922, Validation Accuracy: 0.8590, Loss: 0.0708
Epoch   1 Batch  740/1077 - Train Accuracy: 0.9059, Validati

...

Epoch   4 Batch  105/1077 - Train Accuracy: 0.9594, Validation Accuracy: 0.9247, Loss: 0.0272
Epoch   4 Batch  110/1077 - Train Accuracy: 0.9844, Validation Accuracy: 0.9435, Loss: 0.0315
Epoch   4 Batch  115/1077 - Train Accuracy: 0.9277, Validation Accuracy: 0.9322, Loss: 0.0349
Epoch   4 Batch  120/1077 - Train Accuracy: 0.9352, Validation Accuracy: 0.9460, Loss: 0.0384
Epoch   4 Batch  125/1077 - Train Accuracy: 0.9494, Validation Accuracy: 0.9393, Loss: 0.0372
Epoch   4 Batch  130/1077 - Train Accuracy: 0.9665, Validation Accuracy: 0.9070, Loss: 0.0283
Epoch   4 Batch  135/1077 - Train Accuracy: 0.9346, Validation Accuracy: 0.9158, Loss: 0.0314
Epoch   4 Batch  140/1077 - Train Accuracy: 0.9733, Validation Accuracy: 0.9446, Loss: 0.0211
Epoch   4 Batch  145/1077 - Train Accuracy: 0.9563, Validation Accuracy: 0.9265, Loss: 0.0341
Epoch   4 Batch  150/1077 - Train Accuracy: 0.9546, Validation Accuracy: 0.9315, Loss: 0.0337
Epoch   4 Batch  155/1077 - Train Accuracy: 0.9488, Validati

Epoch   4 Batch  545/1077 - Train Accuracy: 0.9672, Validation Accuracy: 0.9403, Loss: 0.0309
Epoch   4 Batch  550/1077 - Train Accuracy: 0.9492, Validation Accuracy: 0.9400, Loss: 0.0242
Epoch   4 Batch  555/1077 - Train Accuracy: 0.9648, Validation Accuracy: 0.9489, Loss: 0.0193
Epoch   4 Batch  560/1077 - Train Accuracy: 0.9492, Validation Accuracy: 0.9379, Loss: 0.0269
Epoch   4 Batch  565/1077 - Train Accuracy: 0.9661, Validation Accuracy: 0.9297, Loss: 0.0294
Epoch   4 Batch  570/1077 - Train Accuracy: 0.9531, Validation Accuracy: 0.9276, Loss: 0.0387
Epoch   4 Batch  575/1077 - Train Accuracy: 0.9535, Validation Accuracy: 0.9489, Loss: 0.0248
Epoch   4 Batch  580/1077 - Train Accuracy: 0.9379, Validation Accuracy: 0.9560, Loss: 0.0235
Epoch   4 Batch  585/1077 - Train Accuracy: 0.9758, Validation Accuracy: 0.9489, Loss: 0.0195
Epoch   4 Batch  590/1077 - Train Accuracy: 0.9478, Validation Accuracy: 0.9478, Loss: 0.0318
Epoch   4 Batch  595/1077 - Train Accuracy: 0.9766, Validati

Epoch   4 Batch  985/1077 - Train Accuracy: 0.9738, Validation Accuracy: 0.9648, Loss: 0.0218
Epoch   4 Batch  990/1077 - Train Accuracy: 0.9515, Validation Accuracy: 0.9418, Loss: 0.0314
Epoch   4 Batch  995/1077 - Train Accuracy: 0.9669, Validation Accuracy: 0.9332, Loss: 0.0294
Epoch   4 Batch 1000/1077 - Train Accuracy: 0.9449, Validation Accuracy: 0.9329, Loss: 0.0307
Epoch   4 Batch 1005/1077 - Train Accuracy: 0.9512, Validation Accuracy: 0.9212, Loss: 0.0333
Epoch   4 Batch 1010/1077 - Train Accuracy: 0.9668, Validation Accuracy: 0.9332, Loss: 0.0226
Epoch   4 Batch 1015/1077 - Train Accuracy: 0.9465, Validation Accuracy: 0.9453, Loss: 0.0334
Epoch   4 Batch 1020/1077 - Train Accuracy: 0.9672, Validation Accuracy: 0.9503, Loss: 0.0246
Epoch   4 Batch 1025/1077 - Train Accuracy: 0.9617, Validation Accuracy: 0.9414, Loss: 0.0228
Epoch   4 Batch 1030/1077 - Train Accuracy: 0.9621, Validation Accuracy: 0.9460, Loss: 0.0266
Epoch   4 Batch 1035/1077 - Train Accuracy: 0.9777, Validati

Epoch   5 Batch  350/1077 - Train Accuracy: 0.9613, Validation Accuracy: 0.9439, Loss: 0.0336
Epoch   5 Batch  355/1077 - Train Accuracy: 0.9412, Validation Accuracy: 0.9432, Loss: 0.0246
Epoch   5 Batch  360/1077 - Train Accuracy: 0.9488, Validation Accuracy: 0.9528, Loss: 0.0209
Epoch   5 Batch  365/1077 - Train Accuracy: 0.9605, Validation Accuracy: 0.9535, Loss: 0.0214
Epoch   5 Batch  370/1077 - Train Accuracy: 0.9669, Validation Accuracy: 0.9585, Loss: 0.0277
Epoch   5 Batch  375/1077 - Train Accuracy: 0.9556, Validation Accuracy: 0.9549, Loss: 0.0305
Epoch   5 Batch  380/1077 - Train Accuracy: 0.9766, Validation Accuracy: 0.9510, Loss: 0.0237
Epoch   5 Batch  385/1077 - Train Accuracy: 0.9625, Validation Accuracy: 0.9659, Loss: 0.0233
Epoch   5 Batch  390/1077 - Train Accuracy: 0.9500, Validation Accuracy: 0.9389, Loss: 0.0429
Epoch   5 Batch  395/1077 - Train Accuracy: 0.9583, Validation Accuracy: 0.9482, Loss: 0.0257
Epoch   5 Batch  400/1077 - Train Accuracy: 0.9398, Validati

Epoch   5 Batch  790/1077 - Train Accuracy: 0.9250, Validation Accuracy: 0.9371, Loss: 0.0356
Epoch   5 Batch  795/1077 - Train Accuracy: 0.9379, Validation Accuracy: 0.9485, Loss: 0.0329
Epoch   5 Batch  800/1077 - Train Accuracy: 0.9531, Validation Accuracy: 0.9276, Loss: 0.0303
Epoch   5 Batch  805/1077 - Train Accuracy: 0.9383, Validation Accuracy: 0.9382, Loss: 0.0288
Epoch   5 Batch  810/1077 - Train Accuracy: 0.9580, Validation Accuracy: 0.9474, Loss: 0.0213
Epoch   5 Batch  815/1077 - Train Accuracy: 0.9449, Validation Accuracy: 0.9499, Loss: 0.0303
Epoch   5 Batch  820/1077 - Train Accuracy: 0.9609, Validation Accuracy: 0.9343, Loss: 0.0273
Epoch   5 Batch  825/1077 - Train Accuracy: 0.9672, Validation Accuracy: 0.9542, Loss: 0.0121
Epoch   5 Batch  830/1077 - Train Accuracy: 0.9320, Validation Accuracy: 0.9585, Loss: 0.0333
Epoch   5 Batch  835/1077 - Train Accuracy: 0.9809, Validation Accuracy: 0.9482, Loss: 0.0248
Epoch   5 Batch  840/1077 - Train Accuracy: 0.9664, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [23]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    unkonwn_int = vocab_to_int["<UNK>"]
    
    int_word = lambda word : vocab_to_int[word] if word in vocab_to_int else unkonwn_int
    
    return [int_word(word) for word in sentence.lower().split()]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [28]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [213, 51, 21, 89, 121, 104, 144]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [84, 7, 279, 294, 192, 241, 71, 325, 1]
  French Words: il a vu un vieux camion en . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.